In [3]:
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing
from datasource.types import HistoricalApy
from constant.whitelist_lst import WEIGHTED_LST
from helper.historical import load_cached_historical_apys
from allocator.ema import EmaAllocator
from allocator.ma import MaAllocator
from helper.plot import *
import numpy as np
import pandas as pd

weighted_symbols = WEIGHTED_LST

# ? Simple indicator implementations

def calculate_holt_linear_trend(historical_apys: list[HistoricalApy], lookback_period = 2, forecast_period = 1) -> list[Datapoint]:
    forecasts = []
    windows = [historical_apys[:i+lookback_period] for i in range(len(historical_apys) - lookback_period + 1)]
    for window in windows:
        apy_data = np.array([x.apy for x in window])
        model = Holt(apy_data).fit(smoothing_level=0.5, smoothing_trend=0.5, optimized=False)
        forecast = model.forecast(forecast_period)
        latest_epoch = window[-1].epoch
        forecast_epoch = window[-1].epoch + forecast_period
        forecasts.append(Datapoint(latest_epoch, forecast[-1], f"Forecasted APY for epoch {forecast_epoch}"))
    return forecasts

def calculate_holt_winters(historical_apys: list[HistoricalApy], lookback_period = 2, forecast_period = 1) -> list[Datapoint]:
    apys = np.array([x.apy for x in historical_apys])
    df = pd.DataFrame({"apy": apys})
    model = ExponentialSmoothing(df["apy"], trend="add", seasonal="add", seasonal_periods=7).fit()
    df["holtwinters_pred"] = model.fittedvalues
    return df["holtwinters_pred"].tolist()

def calculate_ma(historical_apys: list[HistoricalApy], period = 7) -> list[Datapoint]:
    apys = np.array([x.apy for x in historical_apys])
    df = pd.DataFrame({"apy": apys})
    return df["apy"].rolling(window=period).mean().tolist()

In [4]:
lst_apys_datapoints: list[PlotData] = []
lst_apys_ma: list[PlotData] = []
lst_apys_ema: list[PlotData] = []

for weighted_symbol in weighted_symbols:
    lst = weighted_symbol.symbol
    historical_apys = load_cached_historical_apys(lst)

    lst_apys_datapoints.append(PlotData(lst, [Datapoint(x.epoch, x.apy) for x in historical_apys]))

    emas = EmaAllocator().calculate_ema(historical_apys, 5)
    emas = [0] * (len(historical_apys) - len(emas)) + emas # pad the emas with 0s
    lst_apys_ema.append(PlotData(lst, [Datapoint(x.epoch, y) for x, y in zip(historical_apys, emas)]))

    ma = MaAllocator().calculate_ma(historical_apys, 5)
    ma = [0] * (len(historical_apys) - len(ma)) + ma # pad the ma with 0s
    lst_apys_ma.append(PlotData(lst, [Datapoint(x.epoch, y) for x, y in zip(historical_apys, ma)]))


plots("APY", lst_apys_datapoints)
plots("APY EMA", lst_apys_ema)
plots("APY MA", lst_apys_ma)